<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# DKN : Deep Knowledge-Aware Network for News Recommendation
DKN \[1\] is a deep learning model which incorporates information from knowledge graph for better news recommendation. Specifically, DKN uses TransX \[2\] method for knowledge graph representaion learning, then applies a CNN framework, named KCNN, to combine entity embedding with word embedding and generate a final embedding vector for a news article. CTR prediction is made via an attention-based neural scorer. 

## Properties of DKN:
- DKN is a content-based deep model for CTR prediction rather than traditional ID-based collaborative filtering. 
- It makes use of knowledge entities and common sense in news content via joint learning from semantic-level and knnowledge-level representations of news articles.
- DKN uses an attention module to dynamically calculate a user's aggregated historical representaition.

## Reference environment 
we tested this notebook with two environment settings:
cpu with tensorflow==1.15.2 and gpu with tensorflow==1.15.2 on July 1, 2020.

## Data format:
### DKN takes several files as input as follows:
- training / validation / test files: each line in these files represents one instance. Impressionid is used to evaluate performance within an impression session, so it is only used when evaluating, you can set it to 0 for training data. The format is : <br> 
`[label] [userid] [CandidateNews]%[impressionid] `<br> 
e.g., `1 train_U1 N1%0` <br> 
- user history file: each line in this file represents a users' click history. You need to set his_size parameter in config file, which is the max number of user's click history we use. We will automatically keep the last his_size number of user click history, if user's click history is more than his_size, and we will automatically padding 0 if user's click history less than his_size. the format is : <br> 
`[Userid] [newsid1,newsid2...]`<br>
e.g., `train_U1 N1,N2` <br> 
- document feature file:
It contains the word and entity features of news. News article is represented by (aligned) title words and title entities. To take a quick example, a news title may be : Trump to deliver State of the Union address next week , then the title words value may be CandidateNews:34,45,334,23,12,987,3456,111,456,432 and the title entitie value may be: entity:45,0,0,0,0,0,0,0,0,0. Only the first value of entity vector is non-zero due to the word Trump. The title value and entity value is hashed from 1 to n(n is the number of distinct words or entities). Each feature length should be fixed at k(doc_size papameter), if the number of words in document is more than k, you should truncate the document to k words, and if the number of words in document is less than k, you should padding 0 to the end. 
the format is like: <br> 
`[Newsid] [w1,w2,w3...wk] [e1,e2,e3...ek]`
- word embedding/entity embedding/ context embedding files: These are npy files of pretrained embeddings. After loading, each file is a [n+1,k] two-dimensional matrix, n is the number of words(or entities) of their hash dictionary, k is dimension of the embedding, note that we keep embedding 0 for zero padding. 
In this experiment, we used GloVe\[4\] vectors to initialize the word embedding. We trained entity embedding using TransE\[2\] on knowledge graph and context embedding is the average of the entity's neighbors in the knowledge graph.<br>

## Global settings and imports

In [4]:
import sys
sys.path.append("../../")
from reco_utils.recommender.deeprec.deeprec_utils import *
from reco_utils.recommender.deeprec.models.dkn import *
from reco_utils.recommender.deeprec.io.dkn_iterator import *
import papermill as pm
import time

## Download and load data

In [5]:
data_path = 'data_folder/my/DKN-training-folder'
tag = 'small'
yaml_file = './dkn.yaml' #  os.path.join(data_path, r'../../../../../../dkn.yaml')
train_file = os.path.join(data_path, r'train_{0}.txt'.format(tag))
valid_file = os.path.join(data_path, r'valid_{0}.txt'.format(tag))
test_file = os.path.join(data_path, r'test_{0}.txt'.format(tag))
user_history_file = os.path.join(data_path, r'user_history_{0}.txt'.format(tag))
news_feature_file = os.path.join(data_path, r'../paper_feature.txt')
wordEmb_file = os.path.join(data_path, r'word_embedding.npy')
entityEmb_file = os.path.join(data_path, r'entity_embedding.npy')
contextEmb_file = os.path.join(data_path, r'context_embedding.npy')
infer_embedding_file = os.path.join(data_path, r'infer_embedding.txt')
    

## Create hyper-parameters

In [6]:
epoch=10

In [7]:
hparams = prepare_hparams(yaml_file,
                          news_feature_file = news_feature_file,
                          user_history_file = user_history_file,
                          wordEmb_file=wordEmb_file,
                          entityEmb_file=entityEmb_file,
                          contextEmb_file=contextEmb_file,
                          epochs=epoch,
                          is_clip_norm=True,
                          max_grad_norm=0.5,
                          history_size=20,
                          MODEL_DIR=os.path.join(data_path, 'save_models'),
                          learning_rate=0.002,
                          embed_l2=0.0,
                          layer_l2=0.0,
                          use_entity=True,
                          use_context=True
                         )
print(hparams)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

kg_file=None,user_clicks=None,FEATURE_COUNT=None,FIELD_COUNT=None,data_format=dkn,PAIR_NUM=None,DNN_FIELD_NUM=None,n_user=None,n_item=None,n_user_attr=None,n_item_attr=None,iterator_type=None,SUMMARIES_DIR=None,MODEL_DIR=data_folder/my/DKN-training-folder/save_models,wordEmb_file=data_folder/my/DKN-training-folder/word_embedding.npy,entityEmb_file=data_folder/my/DKN-training-folder/entity_embedding.npy,contextEmb_file=data_folder/my/DKN-training-folder/context_embedding.npy,news_feature_file=data_folder/my/DKN-training-folder/../paper_feature.txt,user_history_file=data_folder/my/DKN-training-folder/user_history_small.txt,use_ent

In [8]:
input_creator = DKNTextIterator

## Train the DKN model

In [9]:
model = DKN(hparams, input_creator)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
#model.load_model(os.path.join(hparams.MODEL_DIR, 'epoch_10'))

In [11]:
t01 = time.time()
print(model.run_eval(valid_file))
t02 = time.time()
print((t02-t01)/60)

{'auc': 0.4883, 'group_auc': 0.4881, 'mean_mrr': 0.4469, 'ndcg@2': 0.3132, 'ndcg@4': 0.5019, 'ndcg@6': 0.582}
0.23990869124730427


In [12]:
model.fit(train_file, test_file)

at epoch 1
train info: logloss loss:0.3308730778246623
eval info: auc:0.8913, group_auc:0.8844, mean_mrr:0.5691, ndcg@2:0.5024, ndcg@4:0.6024, ndcg@6:0.638
at epoch 1 , train time: 134.5 eval time: 21.8
at epoch 2
train info: logloss loss:0.27679864579855007
eval info: auc:0.9033, group_auc:0.8987, mean_mrr:0.5987, ndcg@2:0.5374, ndcg@4:0.6358, ndcg@6:0.6676
at epoch 2 , train time: 132.7 eval time: 21.9
at epoch 3
train info: logloss loss:0.26129320301288783
eval info: auc:0.9091, group_auc:0.9024, mean_mrr:0.6093, ndcg@2:0.555, ndcg@4:0.6487, ndcg@6:0.6769
at epoch 3 , train time: 133.6 eval time: 21.7
at epoch 4
train info: logloss loss:0.2524186024832171
eval info: auc:0.9045, group_auc:0.8997, mean_mrr:0.6058, ndcg@2:0.5461, ndcg@4:0.6418, ndcg@6:0.6733
at epoch 4 , train time: 135.0 eval time: 21.8
at epoch 5
train info: logloss loss:0.25020423358560007
eval info: auc:0.9128, group_auc:0.9069, mean_mrr:0.6247, ndcg@2:0.5732, ndcg@4:0.6607, ndcg@6:0.6903
at epoch 5 , train time: 1

Now we can test again the performance on valid set:

In [13]:
res = model.run_eval(valid_file)
print(res)
pm.record("res", res)

{'auc': 0.9305, 'group_auc': 0.9254, 'mean_mrr': 0.8765, 'ndcg@2': 0.8804, 'ndcg@4': 0.9071, 'ndcg@6': 0.9084}


/home/jialia/.conda/envs/reco_gpu_kdd/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
t01 = time.time()
print(model.run_eval(test_file))
t02 = time.time()
print((t02-t01)/60)

{'auc': 0.9165, 'group_auc': 0.9115, 'mean_mrr': 0.636, 'ndcg@2': 0.5857, 'ndcg@4': 0.6751, 'ndcg@6': 0.7002}
0.367793615659078


## Document embedding inference API
After training, you can get document embedding through this document embedding inference API. The input file format is same with document feature file. The output file fomrat is: `[Newsid] [embedding]`

In [15]:
model.run_get_embedding(news_feature_file, infer_embedding_file)